In [3]:
import os
import sys
import subprocess
import numpy as np
module_path = "/data/luojaa/eukgen/"
sys.path.append(module_path)
from core_functions.helper_functions import fasta_to_dict, dict_to_fasta, filter_by_entropy 
import pandas as pd
from Bio import PDB
from polvir.parseIcarus import *

In [16]:
import os
import subprocess
import numpy as np
import sys
module_path = "/data/luojaa/eukgen/"
sys.path.append(module_path)
from core_functions.helper_functions import fasta_to_dict, dict_to_fasta, filter_by_entropy 
import pandas as pd
# from core_functions.parse_icarus_align import *
# from polvir.parseIcarus import *
from Bio import AlignIO
from Bio.Align import MultipleSeqAlignment
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import random

alignment_root = "/data/luojaa/polvir/alignments/icarus_cropped/"
alignment_file = "icarus_380.muscle3D.entropy_filt.fasta"
msa_path = f"{alignment_root}{alignment_file}"
outdir = "shuffled"
out_root = f"/data/luojaa/polvir/alignments/icarus_cropped/"
alignment = AlignIO.read(msa_path, "fasta")

In [17]:
%%timeit
alignment_np = np.array([list(row.seq) for row in alignment])
ncols = alignment_np.shape[1]
n_replicates = 1

24.9 ms ± 244 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
%%time 


for x in range(50,51):
    nresample = int(ncols * x / 100) 
    for n in range(n_replicates):
        alignment_shuffled = alignment_np.copy()
        
        # round down
        ind_toreplace = set(random.sample(range(ncols), nresample))
        ind_source = random.sample(sorted(set(range(ncols)) - ind_toreplace), nresample)
        for target, source in zip(ind_toreplace, ind_source):
            alignment_shuffled[:, target] = alignment_np[:, source]
            
        # convert back to MSA
        records = []
        # for i in range(alignment_np.shape[0]):
        #     sequence = ''.join(alignment_shuffled[i])
            
        #     # Create sequence record
        #     record = SeqRecord(
        #         Seq(sequence),
        #         id=alignment[0].id,
        #         name=alignment[0].name,
        #         description=alignment[0].description
        #     )
        #     records.append(record)
        # outfile = alignment_file.replace(".fasta", f".x{x}_rep{n}.fasta")
        # AlignIO.write(MultipleSeqAlignment(records), open(f"{out_root}{outfile}", "w"), "fasta")

CPU times: user 1.38 ms, sys: 33 µs, total: 1.41 ms
Wall time: 1.37 ms


## Parse icarus output

In [2]:
testdir = "/data/luojaa/polvir/icarus_out_ED/"
test_seq = "Pokke_SRX802963.109.dc"
# test_seqs = ["ED_3702.AT5G63960", "AA2_NC_009464", "AS3_JAEOSZ010000031", "Mi_TARA_MED_NCLDV_00112_000000000008_2"]
# test_seq = "EZ_5850"
# test_seq = "A3_AE009439"
# test_seq = "AA1_BA000048"
# test_seq = "Mi_TARA_PSW_NCLDV_00051_000000000006_2"
pdb_og = f"/data/luojaa/polvir/af2_structures/{test_seq}.pdb"
pdb_raw = f"{testdir}{test_seq}/icarus_results_on_ED_38727.Pavir.Eb01025.1/models/{test_seq}.raw.pdb"
pdb_renum = f"{testdir}{test_seq}/icarus_results_on_ED_38727.Pavir.Eb01025.1/models/{test_seq}.renum.pdb"
#icarus_out = f"{testdir}{test_seq}/icarus_results_on_ED_38727.Pavir.Eb01025.1/data/{test_seq}.icarus"
struct_align = f"{testdir}{test_seq}/icarus_results_on_ED_38727.Pavir.Eb01025.1/.icarus*/icarus_output/results/PDB1_and_PDB2/summary.txt"


In [3]:
# def parse_alignment_summary(text):
#     # Get first BEST ALIGNMENT section
#     section = text.split("BEST ALIGNMENT")[1].split("Aligned distance")[0]
    
#     # Initialize collectors for each row type
#     collected = {
#         'PUs': [],
#         'ori. pos': [],
#         'connect': [],
#         'QUERY': [],
#         'match': [],
#         'TARGET': [],
#         'dist': [],
#         'ali. pos': []
#     }
    
#     current_type = None
    
#     for line in section.split('\n'):
#         # Skip empty lines
#         if not line.strip():
#             continue
            
#         # Check for row type at the start of line
#         for row_type in collected.keys():
#             if line.strip().startswith(row_type):
#                 current_type = row_type
#                 # Extract the content after the label
#                 content = line.split(':', 1)[1] if ':' in line else line[len(row_type):].strip()
#                 collected[current_type].append(content)
#                 break
        
#         # If we're in a current type and line starts with spaces, it's a continuation
#         if current_type and line.startswith(' ' * 8) and not any(line.strip().startswith(t) for t in collected.keys()):
#             collected[current_type][-1] += line.strip()

#     # Join all lines for each type
#     result = {key: ''.join(values) for key, values in collected.items()}
    
#     return result

In [4]:
alignment_data = parse_alignment_summary(struct_align)


In [5]:
dists = alignment_data["dist"]
query = alignment_data["QUERY"]

In [6]:
dists_qinserts = "".join(list(np.array(list(dists))[(np.array(list(query)) != "-") & (np.array(list(query)) != " ")]))
query_contig = query.replace(" ", "").replace("-","")

In [7]:
# def define_gaps(alignment_data, mindist=4):
#     window_size = 30
#     results = []
#     dists = alignment_data["dist"]
#     query = alignment_data["QUERY"]
#     dists_qinserts = "".join(list(np.array(list(dists))[(np.array(list(query)) != "-") & (np.array(list(query)) != " ")]))
    
#     # For each position in string
#     for i in range(len(dists_qinserts)):
#         pos = dists_qinserts[i]


#         isgap_pos = False
#         if (pos.isdigit() and int(pos) > mindist) or (pos == " "):
#             isgap_pos = True
#         # Check all possible windows containing this position
#         is_part_of_valid_window = False
        
#         # Start checking windows that contain current position
#         start_range = max(0, i - window_size + 1)
#         end_range = min(i+ 1, len(dists_qinserts) - window_size + 1)
#         for start in range(start_range, end_range):
#             window = dists_qinserts[start:start + window_size]
#             count = sum(1 for c in window if c == ' ' or (c.isdigit() and int(c) > mindist))
#             # single possibility of being part of gap = mark as gap
#             if isgap_pos:
#                 if (count / window_size) >= 0.85:
#                     is_part_of_valid_window = True
#                     break
#             # single possibility of not being in a gap = mark as not gap
#             elif (count / window_size) < 0.85:
#                 break
#         results.append(is_part_of_valid_window)
#     # boolean array, true is gap position
#     return results

In [8]:
gaps = define_gaps(alignment_data, 4)
notgaps = [not gap for gap in gaps]
dists_clean = "".join(list(np.array(list(dists_qinserts))[notgaps]))
query_clean = "".join(list(np.array(list(query_contig))[notgaps]))
dists_clean

'121100210000001114767   9              565211123476443 8  54 846 7462211101111110112135  41211100011111001100001120111111012 23343442232110011214730111100000010111111100001122112222111101111101112015 10111111111124211111122211236545651112121  211113524  4 852311111010011112314322223 2 32200000111122110010010000100111211000011111111111010000111111110112221222112111111 1 201334241445   6    6   11223345752513322254121012121001000011 20100001111111242223334444332212231221121011011101101101111111111111112221010012100001100100110110010002 232235621112210011111121111101000000000000000000010123 1211233435761111243       42111113        5 11000000000001112334432221110012 2                  42   21111111100012244200012 3  3   6                00111122111111011111   2  210001111000110000000010000123313 3 3111111221224322111001000101101 3142   6664333110333323332210011111443411222222222323412111002100000112          322111000111111212100112210233111101100121 30                     21111111433423

In [9]:
len(query), len(query_contig), len(query_clean)

(1712, 1615, 1076)

In [10]:
# def map_gaps(bool_arr):
#     if sum(bool_arr) == 0:
#         return [(0,0)] # null gap
#     # Add sentinels 
#     padded = np.r_[False, bool_arr, False]
    
#     # Find transitions
#     transitions = np.where(np.diff(padded))[0]
    
#     # Get ranges
#     return list(zip(transitions[::2], transitions[1::2]))

In [11]:
gap_ranges = map_gaps(gaps)

In [12]:
pdb_renum

'/data/luojaa/polvir/icarus_out_ED/Pokke_SRX802963.109.dc/icarus_results_on_ED_38727.Pavir.Eb01025.1/models/Pokke_SRX802963.109.dc.renum.pdb'

In [13]:
%%capture
output_file = "/data/luojaa/polvir/processing/test.pdb"
parser = PDB.PDBParser()
structure = parser.get_structure("protein", pdb_renum)

# Create a new structure for the spliced portion
new_structure = PDB.Structure.Structure("spliced_protein")

# Iterate through models, chains, and residues
start_number = 5000
count_res = 0
for model in structure:
    new_model = PDB.Model.Model(model.id)
    new_structure.add(new_model)
    for chain in model:
        new_chain = PDB.Chain.Chain(chain.id)
        new_model.add(new_chain)
        # renumber for each chain
        new_resid = start_number
        for residue in chain:
            
            # check if insertion or not
            residue_num = residue.id[1]
        
            if not any(start <= residue_num - 1 < end for start, end in gap_ranges):
                # renum
                residue.id = (residue.id[0], new_resid, residue.id[2])
                new_resid += 1
                new_chain.add(residue)
                count_res += 1
            

# Save the new structure
io = PDB.PDBIO()
io.set_structure(new_structure)
io.save(output_file)

In [102]:
# pdb_renum = f"{testdir}{seq}/icarus_results_on_ED_38727.Pavir.Eb01025.1/models/{seq}.renum.pdb"
test_seq = "Pokke_SRX802963.109.dc"
struct_align = f"{testdir}{test_seq}/icarus_results_on_ED_38727.Pavir.Eb01025.1/.icarus*/icarus_output/results/PDB1_and_PDB2/summary.txt"

#example_text = subprocess.run(f"cat {struct_align}", shell = True, capture_output=True, text=True).stdout
alignment_data = parse_alignment_summary(struct_align)



gaps = define_gaps(alignment_data)
gap_ranges = map_gaps(gaps)

# write fastas
dists = alignment_data["dist"]
query = alignment_data["QUERY"]
dists_qinserts = "".join(list(np.array(list(dists))[(np.array(list(query)) != "-") & (np.array(list(query)) != " ")]))
query_contig = query.replace(" ", "").replace("-","")
notgaps = [not gap for gap in gaps]
dists_clean = "".join(list(np.array(list(dists_qinserts))[notgaps]))
query_clean = "".join(list(np.array(list(query_contig))[notgaps]))


In [14]:
len(query_clean)

1076

In [15]:
count_res

1076

In [16]:
pdb_renum

'/data/luojaa/polvir/icarus_out_ED/Pokke_SRX802963.109.dc/icarus_results_on_ED_38727.Pavir.Eb01025.1/models/Pokke_SRX802963.109.dc.renum.pdb'

In [17]:
%%capture
parser = PDB.PDBParser()
test_seq = "Pokke_SRX802963.109.dc"
# pdb_renum = f"{testdir}{test_seq}/icarus_results_on_ED_38727.Pavir.Eb01025.1/models/{test_seq}.renum.pdb"
struct_align = f"{testdir}{test_seq}/icarus_results_on_ED_38727.Pavir.Eb01025.1/.icarus*/icarus_output/results/PDB1_and_PDB2/summary.txt"
alignment_data = parse_alignment_summary(struct_align)
gaps = define_gaps(alignment_data)
gap_ranges = map_gaps(gaps)

crop_pdb("/data/luojaa/polvir/icarus_out_ED/", "/data/luojaa/polvir/processing/", test_seq, gap_ranges)

In [112]:
%%capture
test_seqs = ["ED_3702.AT5G63960", "AA2_NC_009464", "AS3_JAEOSZ010000031", "Mi_TARA_MED_NCLDV_00112_000000000008_2"]
test_seqs = ["EZ_5762.XP_002673282"]
test_seqs = os.listdir("/data/luojaa/polvir/icarus_out_ED/")
for seq in test_seqs:
    #pdb_og = f"/data/luojaa/polvir/af2_structures/{seq}.pdb"
    #pdb_raw = f"{testdir}{seq}/icarus_results_on_ED_38727.Pavir.Eb01025.1/models/{seq}.raw.pdb"
    pdb_renum = f"{testdir}{seq}/icarus_results_on_ED_38727.Pavir.Eb01025.1/models/{seq}.renum.pdb"
    struct_align = f"{testdir}{seq}/icarus_results_on_ED_38727.Pavir.Eb01025.1/.icarus*/icarus_output/results/PDB1_and_PDB2/summary.txt"
    
    #example_text = subprocess.run(f"cat {struct_align}", shell = True, capture_output=True, text=True).stdout
    alignment_data = parse_alignment_summary(struct_align)
    

    
    gaps = define_gaps(alignment_data)
    gap_ranges = map_gaps(gaps)

    # write fastas
    dists = alignment_data["dist"]
    query = alignment_data["QUERY"]
    dists_qinserts = "".join(list(np.array(list(dists))[(np.array(list(query)) != "-") & (np.array(list(query)) != " ")]))
    query_contig = query.replace(" ", "").replace("-","")
    notgaps = [not gap for gap in gaps]
    dists_clean = "".join(list(np.array(list(dists_qinserts))[notgaps]))
    query_clean = "".join(list(np.array(list(query_contig))[notgaps]))
    outfasta = f"/data/luojaa/polvir/icarus_seqs/{seq}.fasta"
    with open(outfasta,"w") as outfile:
        print(f">{seq}", file=outfile)
        print(query_clean, file=outfile)
    #done writing fastas
    
    output_file = f"/data/luojaa/polvir/icarus_out_ED/{seq}/icarus_results_on_ED_38727.Pavir.Eb01025.1/models/{seq}.cropped.pdb"
    parser = PDB.PDBParser()
    structure = parser.get_structure("protein", pdb_renum)
    
    # Create a new structure for the spliced portion
    new_structure = PDB.Structure.Structure("spliced_protein")
    
    # Iterate through models, chains, and residues
    start_number = 5000
    # renumber for each model
    new_resid = start_number
    nchains = 0
    for model in structure:
        
        print(seq)
        new_model = PDB.Model.Model(model.id)
        new_structure.add(new_model)
        for chain in model:
            nchains += 1
            print("chain")
            new_chain = PDB.Chain.Chain(chain.id)
            new_model.add(new_chain)
            # # renumber for each chain
            # new_resid = start_number
            for residue in chain:
                
                # check if insertion or not
                residue_num = residue.id[1]
            
                if not any(start <= residue_num - 1 < end for start, end in gap_ranges):
                    # renum
                    residue.id = (residue.id[0], new_resid, residue.id[2])
                    new_resid += 1
                    new_chain.add(residue)
    # Save the new structure
    io = PDB.PDBIO()
    io.set_structure(new_structure)
    io.save(output_file)

## compare icarus outputs 

In [114]:
test_seq

'Pokke_SRX802963.109.dc'

In [115]:
# prep files to download locally
file_bases = """
Pokke_GVMAG-M-3300027793-10
EA_29176.XP_003885865
NC_018874
MW412419
EZ_5850
EA_5811
""".strip().split("\n") + [test_seq]
for file in file_bases:
    renum = f"{testdir}{file}/icarus_results_on_ED_38727.Pavir.Eb01025.1/models/{file}.renum.pdb"
    cropped = f"{testdir}{file}/icarus_results_on_ED_38727.Pavir.Eb01025.1/models/{file}.cropped.pdb"
    raw = f"/data/luojaa/polvir/af2_structures/{file}.pdb"
    destination = "/data/luojaa/polvir/processing/icarus_sandbox/"
    os.system(f"cp -t {destination} {renum} {cropped} {raw}")

## rename for muscle3d

In [ ]:
## bash
#find *.pdb -type f -name "*.*.*" -exec sh -c 'echo mv "$1" "$(echo "$1" | sed "s/\./\_/1")"' _ {} \;

## entropy filters

In [131]:
from core_functions.helper_functions import fasta_to_dict, dict_to_fasta, filter_by_entropy 

In [5]:
poly = "/data/luojaa/polvir/alignments/icarus_cropped/icarus_380.muscle_diversified_10.fasta"
poly_dict = fasta_to_dict(file = poly)
poly_filter = filter_by_entropy(poly_dict, .2)
#dict_to_fasta(poly_filter, write_file="/data/luojaa/polvir/alignments/icarus_cropped/icarus_380.muscle_diversified_10.entropy_filt.fasta")

In [18]:
basepath = "/data/luojaa/polvir/alignments/icarus_cropped/"
for f in os.listdir(basepath):
    if f[-6:] == ".fasta" and "usalign" not in f:
        inpath = f"{basepath}{f}"
        f_out = f.replace(".fasta", ".entropy_filt.fasta")
        outpath = f"{basepath}{f_out}"
        poly_dict = fasta_to_dict(file = inpath)
        poly_filter = filter_by_entropy(poly_dict, .2)
        dict_to_fasta(poly_filter, write_file=outpath)

Wrote /data/luojaa/polvir/alignments/icarus_cropped/icarus_380.clustalo.entropy_filt.fasta
Wrote /data/luojaa/polvir/alignments/icarus_cropped/icarus_380.famsa.entropy_filt.fasta
Wrote /data/luojaa/polvir/alignments/icarus_cropped/icarus_380.foldmason_aa.entropy_filt.fasta
Wrote /data/luojaa/polvir/alignments/icarus_cropped/icarus_380.mafft_linsi.entropy_filt.fasta
Wrote /data/luojaa/polvir/alignments/icarus_cropped/icarus_380.muscle3d.entropy_filt.fasta
